In [1]:
import os.path
import numpy as np
import copy
#import networkx as nx
import os
import pickle

In [2]:
arquivo = os.path.join('projeto.csv') 

dataRDD = (sc.textFile(arquivo, 8))

In [3]:
RDD1 = dataRDD.map(lambda l: l.split(',')).map(lambda l: {'label':float(l[0]),'features':[float(l[1]),float(l[2]),l[3]]})
RDD = RDD1.filter(lambda v: v['label']<=1.61)

In [4]:
weights = [.999,.001]
seed = 42
lpRDD, lpTest = RDD.randomSplit(weights, seed)

In [5]:
def contador(lp, feats, splits):
    r = []
    for feat in feats:
        splitss = splits[feat]
        for split in splitss:
            if feat==3:
                r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
            elif feat==0 or feat==1:
                if lp['features'][feat] >= split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
            elif feat==2:                
                if lp['features'][feat] in split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
    return r

In [6]:
def soma(t1,t2):
    return  ((t1[0][0]+t2[0][0],t1[0][1]+t2[0][1]), #((0.668333333, 0), (0.44666944399888886, 0), (1, 0))
             (t1[1][0]+t2[1][0],t1[1][1]+t2[1][1]),
             (t1[2][0]+t2[2][0],t1[2][1]+t2[2][1]))

In [7]:
def calc_stats(t):
    if t[1][2][0] > 0:
        mmenor = t[1][0][0] / t[1][2][0]
        sdmenor = (t[1][1][0] - (t[1][0][0]**2 / t[1][2][0]))/(t[1][2][0])
        cvmenor = np.sqrt(sdmenor)/mmenor*100
        menor = (sdmenor, cvmenor,t[1][2][0])
    else:
        menor = (0, 0, 0)
    
    if t[1][2][1] > 0:    
        mmaior = t[1][0][1] / t[1][2][1]
        sdmaior = (t[1][1][1] - (t[1][0][1]**2 / t[1][2][1]))/(t[1][2][1])
        cvmaior = np.sqrt(sdmaior)/mmaior*100
        maior = (sdmaior, cvmaior,t[1][2][1])
    else:
        maior = (0, 0, 0)
    return (t[0], menor, maior)

In [8]:
def sdreduction(t,c):
    #return ((t[0], c[2][0] - t[1][0]*(t[1][2]/c[2][2]) - t[2][0]*(t[2][2]/c[2][2]))) 
    return ((t[0], c[2][0] - (t[1][2]/c[2][2])*t[1][0] - (t[2][2]/c[2][2])*t[2][0], (t[1][1], t[2][1]))) 

In [9]:
def prepara(t):
    
    return ((t['features'][0],t['features'][0]),(t['features'][1],t['features'][1]), set((t['features'][2],)))
    
def minmax(t1,t2):
    return ((min(t1[0][0],t2[0][0]),max(t1[0][1],t2[0][1])),
            (min(t1[1][0],t2[1][0]),max(t1[1][1],t2[1][1])),
             t1[2].union(t2[2]))

In [10]:
def predict(arvore, lp, i=0):
    if type(arvore[i]) != tuple:
        return (lp['label'], arvore[i])
    
    node = arvore[i]

    if node[0]<2:
        if lp['features'][node[0]]<node[1]:
            return predict(arvore, lp, 2*i+1)
        else:
            return predict(arvore, lp, 2*i+2)
    elif node[0]==2:
        if lp['features'][node[0]] in node[1]:
            return predict(arvore, lp, 2*i+1)
        else:
            return predict(arvore, lp, 2*i+2)
        
    

In [11]:
import time
def tree(lpRDD, maxDepth=1, currDepth=0, st=None,verbose=1, arvore={}, pos=0):
    if st!=None:
        if st[0]<10 or maxDepth<currDepth or st[1]<0:
            media = lpRDD.map(lambda x: x['label']).mean()
            arvore[pos] = media
            #print (st, media, currDepth, pos, lpRDD.count())
            return arvore #[caminho, (direction, media)]    
    
    t0 = time.time()

    (min0,max0),(min1,max1),periods =lpRDD.map(prepara).reduce(minmax)
    
    if verbose >= 3:
        print ( "min-max-uniques:", (min0,max0),(min1,max1),periods)
    if verbose >= 2:
        print ('calc minmax: %.2f s'%(time.time()-t0))
    
    periods = list(periods)

    
    distsplits = [i*(max0-min0)/5. for i in range(1,5)]
    timesplits = [i*(max1-min1)/5. for i in range(1,5)]    
    perisplits = [tuple([periods[j] for j in range(i+1)]) for i in range(len(periods)-1)]

    feats = [0,1,2,3]
    splits = [distsplits,timesplits,perisplits,['full']] 
    
    results = lpRDD.flatMap(lambda lp: contador(lp,feats,splits)).reduceByKey(soma)
    
    if verbose >= 2:
        print ('contador: %.2f s'%(time.time()-t0))
    statistics = results.map(lambda c: calc_stats(c)) #.filter(lambda x: x[0][0]<3)
    
    
    #statistics3 = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]==3)
    
    classe = statistics.filter(lambda x: x[0][0]==3).collect()[0]
    if verbose >= 2:
        print ('estatísticas classe: %.2f s'%(time.time()-t0))
    es = statistics.filter(lambda x: x[0][0]<3).map(lambda x: sdreduction(x,classe)).collect()  
    if verbose >= 2:
        print ('estatísticas dados: %.2f s'%(time.time()-t0))
        
    e = max(es, key = lambda t: t[1]) #atributo / split escolhido
    
    if verbose >= 1:
        print ('atributo escolhido: %.2f s'%(time.time()-t0))
        #print (e)
        
        
    if e[0][0]<2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]<e[0][1])
            if verbose >= 2:
                print ('filtro menor01: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]>e[0][1])   
            if verbose >= 2:
                print ('filtro maior01: %.2f s'%(time.time()-t0))
    elif e[0][0]==2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] in e[0][1])
            if verbose >= 2:
                print ('filtro menor2: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] not in e[0][1])
            if verbose >= 2:
                print ('filtro maior2: %.2f s'%(time.time()-t0))
    
    #caminho.append((direction,e[0]))
    #camenor = copy.deepcopy(caminho)
    #camaior = copy.deepcopy(caminho)
    arvore[pos] = e[0]
    #print (e[0], pos)
    
    #if menorRDD.count()>1:
    amenor = tree(menorRDD, maxDepth, currDepth+1,(e[2][0], e[1]),verbose,arvore,pos=(2*pos+1))
  
    #if maiorRDD.count()>1:
    amaior = tree(maiorRDD, maxDepth, currDepth+1,(e[2][1], e[1]),verbose,arvore,pos=(2*pos+2))
    return arvore

In [12]:
ti=time.time()
arvore = tree(lpRDD,maxDepth=6,verbose=0)
tf = time.time()

In [13]:
print('Tempo total de construção paralelizado (local[4]): %.2f segundos'%(tf-ti))

Tempo total de construção paralelizado (local[4]): 1962.85 segundos


In [14]:
arvore

{0: (1, 0.966),
 1: (2, ('n', 'ep', 'm')),
 2: (2, ('n', 'ep')),
 3: (2, ('n',)),
 4: (0, 14.111904000000001),
 5: (0, 9.257824),
 6: 1.2442886236810111,
 7: (1, 0.7726666666664),
 8: (2, ('m',)),
 9: (1, 0.7726666666664),
 10: (2, ('pm',)),
 11: 1.5427777777799998,
 12: 1.1424757939921832,
 15: (0, 14.102472),
 16: 0.9309187547456215,
 17: (1, 0.7726666666664),
 18: (0, 14.112968),
 19: (0, 11.289280000000002),
 20: (0, 8.14508),
 21: (1, 0.5351111111112001),
 22: (0, 1.4110799999999997),
 31: (1, 0.618),
 32: 0.8019344791580837,
 35: (0, 10.576884000000002),
 36: (0, 10.963128),
 37: (1, 0.7726666666664),
 38: 0.968855450489887,
 39: (1, 0.618),
 40: (1, 0.59577777777776),
 41: 1.5072571054435044,
 42: (2, ('pm',)),
 43: 0.610305316092,
 44: (0, 1.4112920000000002),
 45: 0.0,
 46: (0, 1.4110799999999997),
 63: (0, 11.281128),
 64: (0, 6.352088),
 71: (1, 0.46349999999999997),
 72: (1, 0.4013333333336),
 73: 1.350463656450786,
 74: (0, 1.3329159999999998),
 75: (1, 0.618),
 76: 1.1287

In [15]:
with open('arvore6.pickle', 'wb') as handle:
    pickle.dump(arvore, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [59]:
with open('arvore.pickle', 'rb') as handle:
    arvore = pickle.load(handle)

In [60]:
arvore

{0: (1, 0.966),
 1: (2, ('n', 'ep', 'm')),
 2: (2, ('n', 'ep')),
 3: (2, ('n',)),
 4: (0, 14.111904000000001),
 5: (0, 9.257824),
 6: 1.2442886236810111,
 7: (1, 0.7726666666664),
 8: (2, ('m',)),
 9: (1, 0.7726666666664),
 10: (2, ('pm',)),
 11: 1.5427777777799998,
 12: 1.1424757939921832,
 15: (0, 14.102472),
 16: 0.9309187547456215,
 17: (1, 0.7726666666664),
 18: (0, 14.112968),
 19: (0, 11.289280000000002),
 20: (0, 8.14508),
 21: (1, 0.5351111111112001),
 22: (0, 1.4110799999999997),
 31: 0.8647754257333788,
 32: 0.8019344791580837,
 35: 0.8969222645640487,
 36: 1.052325329327816,
 37: 1.0222386525149663,
 38: 0.968855450489887,
 39: 1.0834150201238646,
 40: 0.9990946081062726,
 41: 1.5072571054435044,
 42: 1.2107425143840282,
 43: 0.610305316092,
 44: 0.9345149068540415,
 45: 0.0,
 46: 0.9939636604896248}

In [16]:
verify = lpTest.map(lambda x: predict(arvore, x)).map(lambda x: (x[0]-x[1])**2).sum()

In [17]:

print(verify)

75.11316218582964


In [11]:
RDD1.map(lambda x: x['label']).stats()

(count: 4198190, mean: 1.23252597825885, stdev: 1.26502390536, max: 16.9144444444, min: 0.483611111111)

In [18]:
RDD1.map(lambda x: x['label']).filter(lambda v: v<=1.61).stats()


(count: 4002221, mean: 1.0162279862856178, stdev: 0.179239465407, max: 1.61, min: 0.483611111111)

In [19]:
RDD1.map(lambda x: x['label']).filter(lambda v: v>1.61).stats()


(count: 195969, mean: 5.649920391319474, stdev: 3.62732854356, max: 16.9144444444, min: 1.61138888889)

In [3]:
(195969/4198190)*100

4.667940231385431

In [326]:

G = nx.DiGraph()
for v,l in arvore.items():
    if type(l)==tuple:
        if type(l[1])!=tuple:
            G.add_node(v,label=(l[0],'{0:.2f}'.format(l[1])))
        else:
            G.add_node(v,label=l)
            
    else:
        G.add_node(v,label='{0:.2f}'.format(l))

for v in G.nodes():
    
    if 2*v+1 in G:
        G.add_edge(v,2*v+1)
    if 2*v+2 in G:
        G.add_edge(v,2*v+2)

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
p=nx.drawing.nx_pydot.to_pydot(G)
p.write_png('example.png')

None
